<a href="https://colab.research.google.com/github/SG-Akshay10/Dynamic_Programming/blob/main/PSO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Particle Swarm Optimization**

### **Question**: 
Design a neural network (the choice of implementation model can be pytorch, tensorflow or the whitebox model) for the data set shared in the ML lab assignment for neural networks. 

* Develop individual code base using following algorithms for weight optimization:
1.	Genetic Algorithm
2.	Cultural Algorithm
3.	Particle Swarm Optimization
4.	Ant Colony Optimization
* 
Data to be uploaded to github
1.	Note on the comparison of performance for the four methods. 
2.	The codebase for all four methods 
3.	The research papers that you have referred to.



### Dependencies and Dataset

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import random

In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Accuracy
from tensorflow.keras.utils import to_categorical

In [3]:
from sklearn import metrics
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler

In [4]:
import os
import torch
from torch import nn
from torch import optim
from torch.utils.data import Dataset, DataLoader, TensorDataset
from torchvision import datasets, transforms

In [5]:
df= pd.read_csv(r"/content/drive/MyDrive/Colab Notebooks/Bank_Personal_Loan_Modelling.csv")

Columns of the dataset : 
* ID: Customer ID
* Age: Customer Age
* Experience: Amount of work experience in years
* Income: Amount of annual income (in thousands)
* Zipcode: Zipcode of where customer lives
* Family: Number of family members
* CCAvg: Average monthly credit card spendings
* Education: Education level (1: Bachelor, 2: Master, 3: Advanced Degree)
* Mortgage: Mortgage of house (in thousands)
* Securities Account: Boolean of whether customer has a securities account
* CD Account: Boolean of whether customer has Certificate of Deposit account
* Online: Boolean of whether customer uses online banking
* CreditCard: Does the customer use credit card issued by the bank?
* Personal Loan: This is the target variable (Binary Classification Problem)

#### Data Cleaning and Feature Engineering

In [6]:
# Deleting Columns which are not necessary
df.drop(["ID"],axis=1,inplace=True)

## Train Test Split

In [7]:
df.shape

(5000, 13)

In [8]:
x = df.iloc[:,:-1].values
y = df.iloc[:,-1].values

In [9]:
x_train,x_test,y_train,y_test = train_test_split(x, y, test_size=0.2, random_state=69)

In [10]:
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

In [11]:
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((4000, 12), (1000, 12), (4000,), (1000,))

# PyTorch 

In [12]:
import torch
from torch.utils.data import DataLoader, TensorDataset

In [13]:
BATCH_SIZE = 1

In [14]:
train_x = torch.from_numpy(x_train).to(torch.float32)
train_y = torch.from_numpy(y_train).to(torch.float32)

In [15]:
train_x.shape, train_y.shape

(torch.Size([4000, 12]), torch.Size([4000]))

In [16]:
data = TensorDataset(train_x,train_y)
data = DataLoader(data,batch_size=BATCH_SIZE,shuffle=True)

## Building Model

In [17]:
class Model(torch.nn.Module):
    
    def __init__(self):
        super(Model,self).__init__()
        
        self.layer1 = torch.nn.Linear(12,16)
        self.layer2 = torch.nn.Linear(16,1)
        self.sigmoid = torch.nn.Sigmoid()
        self.relu = torch.nn.ReLU()
        
    def forward(self, x):
        x = self.layer1(x)
        x = self.relu(x)
        x = self.layer2(x)
        x = self.sigmoid(x)
        return x

# Weight Optimization using Particle Swarm Optimization

In [18]:
#pip install pyswarms

In [19]:
model = Model()
torch.set_grad_enabled(False)
param = np.concatenate([i.numpy().flatten() for i in model.parameters()])
shape = [i.numpy().shape for i in model.parameters()]
size = [i[0]*i[1] if len(i) == 2 else i[0] for i in shape]

print("Dim : ", len(param))
print("Layers Shape : ", shape)
print("Layers Size : ", size)

Dim :  225
Layers Shape :  [(16, 12), (16,), (1, 16), (1,)]
Layers Size :  [192, 16, 16, 1]


In [44]:
def objective_function(particle, model, train_x, train_y,shape = shape,size=size):
    # Reshape the vector to weights and biases dimension
    accuracy = []
    output = []

    for par in particle:
      param = list()
      cum_sum = 0
      for i in range(len(size)):
          array = particle[cum_sum : cum_sum + size[i]]
          array = array.reshape(shape[i])
          cum_sum += size[i]
          param.append(array)
      param = np.array(param, dtype="object")
      output.append(param)
    
    for i in range(len(output)):
        # Copy Weights and Biases
      model = Model()
      for idx, wei in enumerate(model.parameters()):
          wei.data = (torch.tensor(param[idx])).to(torch.float32)

    # Calculate Accuracy
    y_pred = model(train_x)
    y_pred = torch.where(y_pred>=0.5, 1, 0).flatten()
    acc = (y_pred == train_y).sum().float().item() / len(train_x)
    accuracy = 1 - acc # Optimization function aims to reduce the cost so (1 - accuracy) 
    return accuracy

In [ ]:
# Initialize the particles
n_particles = 500
dimensions = len(param)
x_max = 1.0 * np.ones(dimensions)
x_min = -1.0 * x_max
bounds = (x_max,x_min)
particles = np.random.uniform(low=x_min, high=x_max, size=(n_particles, dimensions))
best_fitness = np.inf
best_particle = None

# Initialize the velocities
v_max = 0.1 * (x_max - x_min)
v_min = -v_max
velocities = np.random.uniform(low=v_min, high=v_max, size=(n_particles, dimensions))

# Set the PSO parameters
c1 = 0.6
c2 = 0.3
w = 0.1

# Run the PSO algorithm
max_iter = 100
for i in range(max_iter):
    # Evaluate the fitness of the particles
    fitness = np.array([objective_function(particles[j], model, train_x, train_y) for j in range(n_particles)])

    # Update the global best
    if np.min(fitness) < best_fitness:
        best_fitness = np.min(fitness)
        best_particle = particles[np.argmin(fitness)]

    # Update the velocities and positions of the particles
    r1 = np.random.uniform(size=(n_particles, dimensions))
    r2 = np.random.uniform(size=(n_particles, dimensions))
    vel = w * velocities + c1 * r1 * (particles - particles) + c2 * r2 * (best_particle - particles)
    particles += vel

    # Clip the positions to the bounds
    particles = np.clip(particles, x_min, x_max)

# Extract best_cost and best_parameter
best_cost = best_fitness
best_parameter = best_particle

# Evaluate the performance of the best parameter
best_accuracy = 1 - objective_function(best_parameter, model, train_x, train_y)
print(f"Best Cost : {best_cost}\n Best Parameter : {best_parameter}")
print(f"Best Parameter : {best_accuracy}")

## Testing 

In [ ]:
test_x = torch.from_numpy(x_test).to(torch.float32)
test_y = torch.from_numpy(y_test).to(torch.float32)

In [ ]:
test = TensorDataset(test_x,test_y)
test = DataLoader(test,batch_size=1)

### Classification Report

In [ ]:
best_model = Model()

In [ ]:
y_pred = best_model(test_x)
y_pred = torch.where(y_pred>=0.5, 1, 0).flatten()
print(classification_report(y_pred,test_y))